# Team Assignment #2

Team Sakia - Sakshee Patil and Iara Ravagni

# Importing libraries and loading CSV data file

In [88]:
import pandas as pd
import sqlite3

# Step 1: Load the CSV file into a DataFrame
df = pd.read_csv('data/tips.csv')

# Step 2: Connect to SQLite
con = sqlite3.connect('ta2-sakia.db')
cursor = con.cursor()

# Step 3: Create a table in the SQLite database
df.to_sql('tips', con, if_exists='replace', index=False)

# Step 4: Commit the transaction and close the connection
# con.commit()
# con.close()

df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


# SQL Queries

## Question 1

Retrieve the average tip percentage for each day of the week

## Question 2

Find the maximum and minimum total bull amounts

## Question 3

Count the number of parties for each size

## Question 4

Retrieve the total bill and tip for parties of size 4 or more, where the tip percentage is greater than 15%


## Question 5

Retrieve the total bill, tip amount, and tip percentage for each combination of day and time, sorted by tip percentage in descending order

## Question 6

Find the average tip percentage for each combination of day, time, and smoker status

In [83]:
query = """SELECT day, time, smoker, SUM(tip)/SUM(total_bill)*100 as avg_tip_perc
FROM tips
GROUP BY day, time, smoker"""

result = cursor.execute(query).fetchall()

df_temp = pd.DataFrame(result, columns=["day", "time", 'smoker', 'avg_tip_perc'])
df_temp

,day,time,smoker,avg_tip_perc
0,Fri,Dinner,No,14.298094
1,Fri,Dinner,Yes,15.163245
2,Fri,Lunch,No,18.773467
3,Fri,Lunch,Yes,18.501488
4,Sat,Dinner,No,15.781324
5,Sat,Dinner,Yes,13.514693
6,Sun,Dinner,No,15.448121
7,Sun,Dinner,Yes,14.580606
8,Thur,Dinner,No,15.974441
9,Thur,Lunch,No,15.615392


## Question 7

Retrieve the total bill, tip amount, and tip percentage for each sex, sorted by total bill in descending order, and limit the results to the top 5 records

In [84]:
query = """SELECT sex, total_bill, tip, (tip/total_bill)*100 as tip_perc
FROM tips
ORDER BY total_bill DESC
LIMIT 5"""

result = cursor.execute(query).fetchall()
df_temp = pd.DataFrame(result, columns=["sex", "total_bill", 'tip', 'tip_perc'])
df_temp

,sex,total_bill,tip,tip_perc
0,Male,50.81,10.00,19.681165
1,Male,48.33,9.00,18.621974
2,Male,48.27,6.73,13.942407
3,Male,48.17,5.00,10.379905
4,Male,45.35,3.50,7.717751


## Question 8

Find the maximum and minimum tip percentage for each day and time combination, along with the corresponding total bill and tip amount

In [85]:
query = """SELECT day, time, 
total_bill, 
tip, 
MAX((tip/total_bill)*100) as max_tip, 
MIN((tip/total_bill)*100) as min_tip
FROM tips
GROUP BY day, time;"""

result = cursor.execute(query).fetchall()

df_temp = pd.DataFrame(result, columns=["day", "time", 'total_bill', 'tip', 'max_tip', 'min_tip'])
df_temp

,day,time,total_bill,tip,max_tip,min_tip
0,Fri,Dinner,28.97,3.00,26.348039,10.355540
1,Fri,Lunch,13.42,1.58,25.931446,11.773472
2,Sat,Dinner,32.83,1.17,32.573290,3.563814
3,Sun,Dinner,16.99,1.01,71.034483,5.944673
4,Thur,Dinner,18.78,3.00,15.974441,15.974441
5,Thur,Lunch,18.64,1.36,26.631158,7.296137


## Question 9 

Retrieve the total bill, tip amount, and tip percentage for parties of size 4 or more, where the tip percentage is greater than 15%, and the total bill is between $50 and $100

In [86]:
query = """SELECT size, total_bill, tip, (tip/total_bill)*100 as tip_perc
FROM tips
WHERE size >= 4 
AND (tip/total_bill)*100 >15  
AND total_bill > 50 
AND total_bill < 100"""

result = cursor.execute(query).fetchall()

df_temp = pd.DataFrame(result, columns=["size", "total_bill", 'tip', 'tip_perc'])
df_temp

,size,total_bill,tip,tip_perc


## Question 10

Find the average tip percentage for each combination of day, time, and smoker status, but only include combinations with more than 5 records

In [87]:
query = """SELECT day, time, smoker, (SUM(tip)/SUM(total_bill))*100 as avg_tip_perc
FROM tips
GROUP BY day, time, smoker
HAVING COUNT(*) > 5"""

result = cursor.execute(query).fetchall()

df_temp = pd.DataFrame(result, columns=["day", "time", 'smoker', 'avg_tip_perc'])
df_temp

,day,time,smoker,avg_tip_perc
0,Fri,Dinner,Yes,15.163245
1,Fri,Lunch,Yes,18.501488
2,Sat,Dinner,No,15.781324
3,Sat,Dinner,Yes,13.514693
4,Sun,Dinner,No,15.448121
5,Sun,Dinner,Yes,14.580606
6,Thur,Lunch,No,15.615392
7,Thur,Lunch,Yes,15.788990


## 5 additional queries


Insert an Additional Row/Rows into the Database

In [94]:
print("Original number of rows in Dataset: ", len(df))
cursor.execute("""INSERT INTO tips VALUES (30.1, 4, "female", "No",	"Thu", "Dinner", 2)""")

many_rows = [
    (70.1, 4, 'female', 'No', 'Thu', 'Dinner', 4),
    (50.7, 5.6, 'male', 'No', 'Fri', 'Dinner', 3),
    (35.9, 4.6, 'female', 'Yes', 'Sat', 'Lunch', 2)]

# using field placeholders
result = cursor.executemany("""INSERT INTO tips VALUES (?, ?, ?, ?, ?, ?, ?)""", many_rows)

df_temp = pd.DataFrame(result)
con.commit()

# Get the updated number of rows
df_temp = pd.read_sql_query("SELECT * FROM tips", con)
print("Modified number of rows in Dataset: ", len(df_temp))

Original number of rows in Dataset:  244
Modified number of rows in Dataset:  253


Exploring Subquerying and Nested Queries: Find all sex and smoker values for tips that are greater than the average tip value:

In [95]:
query ="""SELECT sex, smoker
FROM tips
WHERE tip > (
SELECT AVG(tip)
FROM tips
)
"""

result = cursor.execute(query).fetchall()
print(result)

[('Male', 'No'), ('Male', 'No'), ('Female', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Female', 'No'), ('Male', 'No'), ('Male', 'No'), ('Female', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Female', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Female', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'Yes'), ('Male', 'Yes'), ('Male', 'No'), ('Female', 'Yes'), ('Female', 'Yes'), ('Male', 'Yes'), ('Male', 'No'), ('Male', 'No'), ('Male', 'Yes'), ('Female', 'No'), ('Male', 'No'), ('Male', 'No'), ('Male', 'No'), ('Female', 'Yes'), ('Female', 'No'), ('Male', 'Yes'), ('Male', 'Yes'), ('Female', 'Yes'), ('Female', 'No'), ('Male', 'Yes'), ('Male', 'Yes'), ('Male', 'No'), ('Female', 'Yes'), ('Male', 'No'), ('Female', 'No'), ('Female', 'No'), ('Male', 'No'), ('Female', 'No'), ('Female', 'No'), ('Female', 'No'), ('Male', 'No'), ('Male', 'No'), ('Fema

Creating Indexes

In [97]:
res = cursor.execute("""CREATE INDEX IF NOT EXISTS reservation_no ON tips(total_bill)""")
indexes = cursor.execute("PRAGMA index_list('tips')").fetchall()

print(indexes)

[(0, 'reservation_no', 0, 'c', 0)]


Analyzing Query Performance

In [60]:
# SQLite provides the EXPLAIN QUERY PLAN command, which helps analyze the execution plan of a query and identify areas for optimization.
query = "SELECT * FROM tips WHERE total_bill > 50"
res = cursor.execute("EXPLAIN QUERY PLAN " + query)
data = res.fetchall()
print(data)

[(3, 0, 0, 'SEARCH tips USING INDEX reservation_srno (total_bill>?)')]


Exploring the Self-Join Operation by analyzing tipping patterns between men and women. 

Joins the tips table with itself to compare tips for different sex while matching on other attributes like size, day, smoker, and time.

In [99]:
query = """SELECT t1.sex AS sex1, t1.tip AS tip1, t2.sex AS sex2, t2.tip AS tip2
FROM tips t1
INNER JOIN tips t2
ON t1.size = t2.size
AND t1.day = t2.day
AND t1.smoker = t2.smoker
AND t1.time = t2.time
AND t1.sex != t2.sex"""


result = cursor.execute(query).fetchall()
df_temp = pd.DataFrame(result, columns=["sex 1", "tip 1", 'sex 2', 'tip 2'])
df_temp

,sex 1,tip 1,sex 2,tip 2
0,Female,1.01,Male,1.32
1,Female,1.01,Male,1.56
2,Female,1.01,Male,1.57
3,Female,1.01,Male,1.71
4,Female,1.01,Male,1.96
...,...,...,...,...
1873,Male,1.75,Female,2.47
1874,Male,1.75,Female,2.75
1875,Male,1.75,Female,3.00
1876,Male,1.75,Female,3.00


## UPDATE

It was determined that there was an error in the database. Please update the record that corresponds to id=10 and set smoker to Yes.

## DELETE

Delete records from the database that have a total bill that is less than $10.

In [102]:
query = """DELETE FROM tips
WHERE total_bill < 10"""
cursor.execute(query)

In [103]:
con.close()